In [ ]:
# 是否已整理好数据集
org = True
if not org:
    import os
    import shutil

    def reorg_cifar10_data(data_dir, label_file, train_dir, input_dir, valid_ratio):
        with open(os.path.join(data_dir, label_file), 'r') as f:
            lines = f.readlines()[1:]
            tokens = [l.rstrip().split(',') for l in lines]
            idx_label = dict(((int(idx), label) for idx, label in tokens))
        labels = set(idx_label.values())
        num_train = len(os.listdir(os.path.join(data_dir, train_dir)))
        num_train_tuning = int(num_train * (1 - valid_ratio))
        assert 0 < num_train_tuning < num_train
        num_train_tuning_per_label = num_train_tuning // len(labels)
        label_count = dict()
        def mkdir_if_not_exist(path):
            if not os.path.exists(os.path.join(*path)):
                os.makedirs(os.path.join(*path))
        for train_file in os.listdir(os.path.join(data_dir, train_dir)):
            idx = int(train_file.split('.')[0])
            label = idx_label[idx]
            mkdir_if_not_exist([data_dir, input_dir, 'train_valid', label])
            shutil.copy(os.path.join(data_dir, train_dir, train_file),os.path.join(data_dir, input_dir, 'train_valid', label))
            if label not in label_count or label_count[label] < num_train_tuning_per_label:
                mkdir_if_not_exist([data_dir, input_dir, 'train', label])
                shutil.copy(os.path.join(data_dir, train_dir, train_file),os.path.join(data_dir, input_dir, 'train', label))
                label_count[label] = label_count.get(label, 0) + 1
            else:
                mkdir_if_not_exist([data_dir, input_dir, 'valid', label])
                shutil.copy(os.path.join(data_dir, train_dir, train_file),os.path.join(data_dir, input_dir, 'valid', label))


    train_dir = 'train'

    data_dir = '/Users/Sinyer/Desktop/kaggle'
    label_file = 'trainLabels.csv'
    input_dir = 'train_valid_test'
    valid_ratio = 0.1
    reorg_cifar10_data(data_dir, label_file, train_dir, input_dir, valid_ratio)

In [ ]:
import mxnet as mx
from mxnet import init, gluon, nd, autograd, image
from mxnet.gluon import nn
from mxnet.gluon.data import vision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
ctx = mx.gpu()
data_dir = '/Users/Sinyer/Desktop/kaggle'
label_file = 'trainLabels.csv'
input_dir = 'train_valid_test'

input_str = data_dir + '/' + input_dir + '/'

train_valid_ds = vision.ImageFolderDataset(input_str + 'train_valid', flag=1)

In [ ]:
ctx = mx.cpu()
def mesuare_softmax_sum(preds, weight_list):
    output = nd.softmax(preds[0], axis=1) * weight_list[0]
    for i in range(1, len(preds)):
        output = output + nd.softmax(preds[i], axis=1) * weight_list[i]
    preds = output.argmax(axis=1).astype(int).asnumpy() % 10
    return preds

model_list = ['resnet164_e255_focal_clip', 'res164__2_e255_focal_clip_all_data', 'resnet164_e300','resnet164_e0-255_focal_clip',
               'shelock_densenet_orign',
              'shelock_resnet_orign', 'wrn', 'wrn']
weight_list = [0.9535, 0.9540, 0.95270, 0.95, 0.9539, 0.95, 0.9596, 0.9596]
preds = []
for result_name in model_list:
    preds.append(nd.load("/Users/Sinyer/Desktop/kaggle/"+result_name)[0].as_in_context(ctx))

In [ ]:
preds = mesuare_softmax_sum(preds, weight_list)

sorted_ids = list(range(1, 300000 + 1))
sorted_ids.sort(key=lambda x: str(x))

df = pd.DataFrame({'id': sorted_ids, 'label': preds})
df['label'] = df['label'].apply(lambda x: train_valid_ds.synsets[x])
df.to_csv('/Users/Sinyer/Desktop/ensemble.csv', index=False)